In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4
#import thread

In [ ]:
#this is the training data for our models.
filepath='/home/vedanshi/Documents/DSML/IMD work/HARv2_d10km_d_2d_prcp_nc_2023.nc'
data=xr.open_dataset(filepath,engine='netcdf4')
data

In [ ]:
data.lon.values

In [ ]:
data.prcp_nc.max()

In [ ]:
data.prcp_nc[102].T.plot.surface()

In [ ]:
data.prcp_nc[0, :, :].T.plot.surface()
data.prcp_nc[0].max()

visualizing latitude and longitude as south_north and west_east

In [ ]:
print(len(data.lon.values))
print(len(data.lat.values))
df1=pd.DataFrame(data.lon)
df1

In [ ]:
#checking for duplicates 
print(sum(df1.duplicated()))

In [ ]:
df1.describe()

In [ ]:
df2=pd.DataFrame(data.lat)
df2

In [ ]:
#checking for duplicated
print(sum(df2.duplicated()))

In [ ]:
df2.describe()

extracting latitude and longitude values for day one, i.e these values represent the data recorded on respective lon and lat on 1st day, 
#### Plan :
- make the method just for 1 day and then itrate over for the next 365 days.
- then interpolate for that day.

In [ ]:
new_lat = data.lat[:,0].values
latitude = np.array(new_lat)
# Flatten the 2D array to a 1D array
#flattened_lat = new_lat.flatten()
# Iterate over the flattened array
#for lat in flattened_lat:
 #   latitude.append(lat)
df3=pd.DataFrame(latitude)
df3

In [ ]:
new_lon = data.lon[0,:].values
longitude = np.array(new_lon)
# Flatten the 2D array to a 1D array
#flattened_lon = new_lon.flatten()
# Iterate over the flattened array
#for lon in flattened_lon:
 #   longitude.append(lon)
df4=pd.DataFrame(longitude)
df4

now i have to extract the precipitaion values in mm somehow form the prcp_nc variable.

In [ ]:
import numpy as np
# Extracting precipitation data directly as a NumPy array
prcp_values = data['prcp_nc'][0, :, :].values
# Flatten the 2D array to a 1D array
prcp_values_flat = prcp_values.flatten()
# Now, prcp_values_flat contains all precipitation values as a 1D array
print(prcp_values_flat.max()==data.prcp_nc[0].max())
print((len(latitude)*len(longitude)==len(prcp_values_flat)))
print(len(prcp_values_flat))
print(len(latitude)*len(longitude))
prcp_values_flat.max()
# We have now gotten all the prcp values, now we have to fill them in a dataframe, but the dimenssions dont match 

In [ ]:
x = prcp_values = data['prcp_nc'].shape
x

Making a function of the above process so as it can be used when we need to extract the infromation of the remaining days.

In [ ]:
# i is the index at which all the data is to be exctracted, we will the above data only.
def extraction(i):
    lati=data.lat[:,i]
    lat=np.array(lati)
    longi=data.lon[i,:]
    lon=np.array(longi)
    prcp=data['prcp_nc'][i,:,:].values
    prcp_falt=prcp.flatten()
    return prcp_falt,lon,lat

### Maping of precipitaion vlaues.

In [ ]:
indexr=[]
for i in range(252):
    indexr.append(i)
indexr

In [ ]:
# Create a list of column labels based on the length of latitude
columns_labels = list(range(len(longitude)))
# Create an empty DataFrame with the correct number of columns
temp_df = pd.DataFrame(index=indexr, columns=columns_labels)
print(len(temp_df))
temp_df


In [ ]:
#populating the data frame.
for i in range(381):
    for j in range(252):
        temp_df[i][j]=prcp_values_flat[i*j]
temp_df.index=latitude
temp_df.columns=longitude
temp_df

In [ ]:
import matplotlib.pyplot as plt

# Define latitude and longitude ranges
lat_range = np.array(range(0, 91, 10))  # Positive latitude range (0 to 90)
lon_range = np.array(range(0, 181, 10))  # Positive longitude range (0 to 180)

# Plot the grid without any data
plt.figure(figsize=(10, 6))
plt.grid(True)  # Plot grid lines
plt.xticks(lon_range)  # Set x-axis ticks to longitude values
plt.yticks(lat_range)  # Set y-axis ticks to latitude values
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Grid without Data (Positive Latitude and Longitude)')

# Show the plot
plt.show()

In [ ]:
# create a 2D grid of points for the HAR data
har_lat, har_lon = np.meshgrid(latitude, longitude)
len(har_lon)

In [ ]:
inter=pd.read_excel(r'/home/vedanshi/Documents/DSML/IMD work/avalanche_glacier sites.xlsx')
inter.columns=[0,1,2,3,4,5,6,7,8,9,10]
inter

In [ ]:
to_be_found_lat=inter[1:][0]
to_be_found_lon=inter[1:][1]
to_be_found_lat

In [ ]:
to_be_found_lon

# making grid.

x_values and y_values are the lat_renge and lon_range.

In [ ]:
map_grid=np.empty((252,381),dtype=object)
# 252 - lat values.
# 381 - lon values. 

In [ ]:
# function to calculate grid parameters
def calculate_grid_parameters(coord):
    d = np.diff(coord)[0] / 2
    print(d)
    cell_boundaries = np.concatenate([[coord[0] - d], coord + d])
    return cell_boundaries

In [ ]:
# function to create grid cell taking arguments from x_values and y_values
def initialize_and_populate_grid(x_values, y_values): #x_values and y_value = latitude, longitude.
    ngrid_cell = len(x_values) * len(y_values)
    grid_cell = [{} for i in range(ngrid_cell)]
    dy = y_values[1] - y_values[0]
    k = 0
    
    latcell = calculate_grid_parameters(x_values)
    loncell = calculate_grid_parameters(y_values)
    
    for i, lat1  in enumerate(x_values):
        for j, lon1 in enumerate(y_values):
            x_coords = [round(latcell[i], 2), round(latcell[i], 2), round(latcell[i+1], 2), round(latcell[i+1], 2), round(latcell[i], 2)]
            y_coords = [round(loncell[j], 2), round(loncell[j+1], 2), round(loncell[j+1], 2), round(loncell[j], 2), round(loncell[j], 2)] if dy > 0 else \
                       [round(loncell[j+1], 2), round(loncell[j], 2), round(loncell[j], 2), round(loncell[j+1], 2), round(loncell[j+1], 2)]
            
            grid_cell[k] = {'X': x_coords, 'Y': y_coords, 'Xc': round(lat1, 2), 'Yc': round(lon1, 2)}
            k += 1
    return grid_cell

In [ ]:
if __name__ == "__main__":
    grid_cell = initialize_and_populate_grid(latitude, longitude)
grid_cell

In [ ]:
temp_inter=[]
for i in range(1,21):
    interp=[to_be_found_lat[i],to_be_found_lon[i]]
    temp_inter.append(interp)
to_be_inter=pd.DataFrame(temp_inter)
to_be_inter

In [ ]:
# Flatten the arrays to use in griddata
prcp_points = np.column_stack((har_lat.ravel(), har_lon.ravel()))
prcp=prcp_values_flat.ravel()

In [ ]:
print("Shape of prcp_points:", prcp_points.shape)
print("Shape of prcp_values_flat:", prcp_values_flat.shape)
print("Shape of to_be_inter:", to_be_inter.shape)


## Ineterpolating values

In [ ]:
roundoff_lat=latitude.round(decimals=1)
roundoff_lon=longitude.round(decimals=1)
roundoff_lat

In [ ]:
roundoff_lon

the values rounded off above could be used to extrac the vlaues of y_true.

In [ ]:
def accuracy(y_true,y_interpolated):
    # Calculate MAE
    mae = np.mean(np.abs(y_interpolated - y_true))
    # Calculate MSE
    mse = np.mean((y_interpolated - y_true)**2)
    # Calculate RMSE
    rmse = np.sqrt(mse)
    # Calculate R^2 score
    mean_true = np.mean(y_true)
    r2 = 1 - np.sum((y_true - y_interpolated)**2) / np.sum((y_true - mean_true)**2)
    print("MAE:", mae)
    print("MSE:", mse)
    print("RMSE:", rmse)
    print("R^2 Score:", r2)


In [ ]:
def interpolate(coords,method_used):
    #entered coords must be upto accurcay of only 1 digit after the decimal.
    import scipy.interpolate as sci
    if method_used=='linear' or method_used=='bilinear':
        if method_used=='bilinear':
            return sci.griddata(prcp_points,prcp_values_flat,coords,method='nearest')
        else:
            return sci.griddata(prcp_points,prcp_values_flat,coords,method='linear')
    else :
            inter=sci.NearestNDInterpolator(prcp_points,prcp_values_flat,rescale=False,tree_options=None)
            return inter(coords)

In [ ]:
interpolate(to_be_inter,'bilinear')

In [ ]:
interpolate(to_be_inter,'linear')

In [ ]:
interpolate(to_be_inter,'knn')